In [11]:
import serial

In [20]:
ser =serial.Serial('COM4', 9600, parity=serial.PARITY_NONE,
                   stopbits=serial.STOPBITS_ONE, timeout=5)


In [21]:

def calcCRC(cmnd):
    # cmnd is a byte array containing the command ASCII string; example: cmnd="Sinv2.000"
    # an unsigned 32 bit integer is returned to the calling program
    # only the lower 16 bits contain the crc

    crc = 0xffff # initialize crc to hex value 0xffff
    
    for character in cmnd: # this for loop starts with ASCCII 'S' and loops through to the last ASCII '0'
        hex_char = (int(ord(character)))
        #hex_char = character
        crc=crc^(hex_char*0x0100) # the ASCII value is times by 0x0100 first then XORED to the current crc value
        #for(j=0; j<8; j++) # the crc is hashed 8 times with this for loop
        j = 0
        for j in range(0,8):
            # if the 15th bit is set (tested by ANDING with hex 0x8000 and testing for 0x8000 result) 
            # then crc is shifted left one bit (same as times 2) XORED with hex 0x1021 and ANDED to 
            # hex 0xffff to limit the crc to lower 16 bits. If the 15th bit is not set then the crc 
            # is shifted left one bit and ANDED with hex 0xffff to limit the crc to lower 16 bits.
            if((crc&0x8000)==0x8000):
                crc=((crc<<1)^0x1021)&0xffff
            else:
                crc=(crc<<1)&0xffff
            # end of j loop
        # end of i loop
    # There are some crc values that are not allowed, 0x00 and 0x0d

    # These are byte values so the high byte and the low byte of the crc must be checked and incremented if 
        # the bytes are either 0x00 0r 0x0d
    if((crc&0xff00)==0x0d00):
        crc +=0x0100
    if((crc&0x00ff)==0x000d):
        crc +=0x0001
    if((crc&0xff00)==0x0000):
        crc +=0x0100
    if((crc&0x00ff)==0x0000):
        crc +=0x0001
    
    print(hex(crc))
    return crc.to_bytes(2, 'big')

    # If the string Sinv2.000 is sent through this routine the crc = 0x8f55
    # The complete command "Sinv2.000" will look like this in hex: 
        # 0x53 0x69 0x6E 0x76 0x32 0x2e 0x30 0x30 0x30 0x8f 0x55 0x0d

In [22]:
calcCRC("Sinv2.000")

0x8f55


b'\x8fU'

In [23]:
calcCRC("!StrmOn")
calcCRC("!StrmOff")

0xeb10
0xd98c


b'\xd9\x8c'

In [36]:
def cmd_controller(ser, cmd):
    crc = calcCRC(cmd)
    cmd = cmd.encode() + (crc) + b'\r'
    print(cmd)
    ser.write(cmd)
    #ser_rsp = ser.read(200)
    ser_rsp  = ser.read_until(b'\r')
    print("Output from MFC Controller cmd with repr(): " + repr(ser_rsp))
    print("resp CRC", calcCRC(ser_rsp[:-3].decode()))
    return(ser_rsp)

In [37]:
ser.flush()
ser.flushInput()
cmd_controller(ser, '!StrmEcho')
#cmd_controller(ser, '!FLOWwSETPOINT')
#ser.flushInput()
#ser.write(b"SERIAL_NUMBER\r")
#ser.read(1000)

cmd_controller(ser, '?Srnm')
cmd_controller(ser, '?Gasi')
cmd_controller(ser, '!Sinv42.00')


0xeb49
b'!StrmEcho\xebI\r'
Output from MFC Controller cmd with repr(): b'StrmEcho\x8e\xda\r'
0x8eda
resp CRC b'\x8e\xda'
0xb5ba
b'?Srnm\xb5\xba\r'
Output from MFC Controller cmd with repr(): b'Srnm150730\xde\xa3\r'
0xdea3
resp CRC b'\xde\xa3'
0x4b74
b'?GasiKt\r'
Output from MFC Controller cmd with repr(): b'Gasi2n-\r'
0x6e2d
resp CRC b'n-'
0xe124
b'!Sinv42.00\xe1$\r'
Output from MFC Controller cmd with repr(): b'Sinv42.000\xb8Y\r'
0xb859
resp CRC b'\xb8Y'


b'Sinv42.000\xb8Y\r'

In [65]:
ser.close()